# Data Preparation

In [173]:
import psycopg2
import pandas as pd
import numpy as np
import dbpass
pd.set_option('max_colwidth',0)

### Connecting to the Database

In [174]:
DBNAME = "opportunity_youth"
#conn = psycopg2.connect(dbname=DBNAME, user="postgres")
conn = psycopg2.connect(dbname=DBNAME, user="postgres", password=dbpass.postgrepass())

### Query

We pull the data for the PUMAs (separate regions studied, defined by population and the edges of census tracts) that we are interested in.  We need employment information, education information, and ages.  We pull the data for everyone, not just opportunity youth so we can compare their prevalence to total population.

##### Choice of Regions

We decided to use 11610 - 11615 because on the PUMA_names look up table, they all say they are King Country South somewhere.  We also included 11604 and 11605 because those are clearly south of the City of Seattle.  As a long time Seattle resident, I know that the regions south of downtown tend to have lower property values, lower average incomes, and a greater representation of people of color.  To get a good picture of opportunity youth from this area, we needed to include those regions.  Also, they are included in the 'Opportunity Youth in the Road Map Project Region' report.

##### Sample Weights

Finally, we pull the sample weights, since only a fraction of the population was polled for this dataset.

In [175]:
query = """
SELECT esr, schl education_attained, fschp enrollment_status, agep age, pwgtp sample_weight
FROM pums_2017
where puma between '11610' and '11615'
or puma = '11604'
"""
df = pd.read_sql(query, conn)

In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39202 entries, 0 to 39201
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   esr                 31824 non-null  object 
 1   education_attained  37891 non-null  object 
 2   enrollment_status   39202 non-null  object 
 3   age                 39202 non-null  float64
 4   sample_weight       39202 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.5+ MB


In [177]:
df.esr.fillna('3', inplace = True)

In [178]:
df[df.esr == '3'].count()

esr                   8496
education_attained    7185
enrollment_status     8496
age                   8496
sample_weight         8496
dtype: int64

#### Data Cleaning

We see that esr (employment recode)

##### Employment

For our employment data we want as complete a dataset as possible, so set missing values in esr to '3' with the assumption that missing values represent non-employed samples.

We made this choice for 2 reasons:

1. We assume folks are more likely to report that they are employed than if they are not.
2. We want to avoid under-countring our opportunity youths.

##### Education Attained

For this column it's less clear what missing values might represent.  There are many fewer missing values here, and less chance of severely undercounting our opportunity youth.  Here we choose to take
the mode of the column, representing the most common values.

In [179]:
# Create a new column and fill it with 1s, representing 'employed'.  This initiates our employed column.
df['employed'] = 1

# First we find the unemployed amoung the cow and esr features.  We set those samples to 0, representing 'unemployed'.
df.loc[df.esr == '3','employed'] = 0

# Now we set missing values in education_attained to 0, since our decode says those samples should be less than 3 years old anyway.

df['education_attained'].fillna(0,inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39202 entries, 0 to 39201
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   esr                 39202 non-null  object 
 1   education_attained  39202 non-null  object 
 2   enrollment_status   39202 non-null  object 
 3   age                 39202 non-null  float64
 4   sample_weight       39202 non-null  float64
 5   employed            39202 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1.8+ MB


In [180]:
df.drop(columns = ['esr'], inplace = True)

We've filled our missing values.

In [181]:
df.describe()

,age,sample_weight,employed
count,39202.000000,39202.000000,39202.000000
mean,39.715397,22.646396,0.783276
std,22.926068,16.246238,0.412018
min,0.000000,1.000000,0.000000
25%,21.000000,13.000000,1.000000
50%,40.000000,18.000000,1.000000
75%,58.000000,25.000000,1.000000
max,94.000000,267.000000,1.000000


In [182]:
df.groupby('age').count()

,education_attained,enrollment_status,sample_weight,employed
age,,,,
0.0,422,422,422,422
1.0,447,447,447,447
2.0,442,442,442,442
3.0,479,479,479,479
4.0,476,476,476,476
...,...,...,...,...
87.0,101,101,101,101
88.0,101,101,101,101
89.0,72,72,72,72


#### Age == 0

It's tempting to think age == 0 might be a placeholder, but when I look at the distribution, it seems in line with other young children.  I'm going to assume it means an infant and leave them.

In [183]:
df['education_attained'].unique()

array(['05', '06', '04', '11', '22', '19', '16', '20', '21', '18', '23',
       '14', '13', '07', '17', '08', '09', '12', '01', '15', '02', 0,
       '10', '03', '24'], dtype=object)

#### Education Attained

These are all valid values.

In [184]:
df.head()

,education_attained,enrollment_status,age,sample_weight,employed
0,05,0,40.0,90.0,1
1,06,0,11.0,78.0,0
2,04,0,9.0,60.0,0
3,11,0,48.0,109.0,1
4,11,0,48.0,108.0,1


In [185]:
df.to_csv('full_database.csv')

In [186]:
conn.close()